# Capstone Project
## Neural translation model
### Instructions

In this notebook, you will create a neural network that translates from English to German. You will use concepts from throughout this course, including building more flexible model architectures, freezing layers, data processing pipeline and sequence modelling.

This project is peer-assessed. Within this notebook you will find instructions in each section for how to complete the project. Pay close attention to the instructions as the peer review will be carried out according to a grading rubric that checks key parts of the project instructions. Feel free to add extra cells into the notebook as required.

### How to submit

When you have completed the Capstone project notebook, you will submit a pdf of the notebook for peer review. First ensure that the notebook has been fully executed from beginning to end, and all of the cell outputs are visible. This is important, as the grading rubric depends on the reviewer being able to view the outputs of your notebook. Save the notebook as a pdf (File -> Download as -> PDF via LaTeX). You should then submit this pdf for review.

### Let's get started!

We'll start by running some imports, and loading the dataset. For this project you are free to make further imports throughout the notebook as you wish. 

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import unicodedata
import re
from IPython.display import Image
# for splitting sentences 
import re
# for finding random samples
import random
from tensorflow.keras.layers import Layer, concatenate, Input, Masking, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np


![Flags overview image](data/germany_uk_flags.png)

For the capstone project, you will use a language dataset from http://www.manythings.org/anki/ to build a neural translation model. This dataset consists of over 200,000 pairs of sentences in English and German. In order to make the training quicker, we will restrict to our dataset to 20,000 pairs. Feel free to change this if you wish - the size of the dataset used is not part of the grading rubric.

Your goal is to develop a neural translation model from English to German, making use of a pre-trained English word embedding module.

In [2]:
# Run this cell to load the dataset

NUM_EXAMPLES = 20000
data_examples = []
with open('data/deu.txt', 'r', encoding='utf8') as f:
    for line in f.readlines():
        if len(data_examples) < NUM_EXAMPLES:
            data_examples.append(line)
        else:
            break
            

In [3]:
# These functions preprocess English and German sentences

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"ü", 'ue', sentence)
    sentence = re.sub(r"ä", 'ae', sentence)
    sentence = re.sub(r"ö", 'oe', sentence)
    sentence = re.sub(r'ß', 'ss', sentence)
    
    sentence = unicode_to_ascii(sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-z?.!,']+", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    return sentence.strip()

#### The custom translation model
The following is a schematic of the custom translation model architecture you will develop in this project.

![Model Schematic](data/neural_translation_model.png)

Key:
![Model key](data/neural_translation_model_key.png)

The custom model consists of an encoder RNN and a decoder RNN. The encoder takes words of an English sentence as input, and uses a pre-trained word embedding to embed the words into a 128-dimensional space. To indicate the end of the input sentence, a special end token (in the same 128-dimensional space) is passed in as an input. This token is a TensorFlow Variable that is learned in the training phase (unlike the pre-trained word embedding, which is frozen).

The decoder RNN takes the internal state of the encoder network as its initial state. A start token is passed in as the first input, which is embedded using a learned German word embedding. The decoder RNN then makes a prediction for the next German word, which during inference is then passed in as the following input, and this process is repeated until the special `<end>` token is emitted from the decoder.

## 1. Text preprocessing
* Create separate lists of English and German sentences, and preprocess them using the `preprocess_sentence` function provided for you above.
* Add a special `"<start>"` and `"<end>"` token to the beginning and end of every German sentence.
* Use the Tokenizer class from the `tf.keras.preprocessing.text` module to tokenize the German sentences, ensuring that no character filters are applied. _Hint: use the Tokenizer's "filter" keyword argument._
* Print out at least 5 randomly chosen examples of (preprocessed) English and German sentence pairs. For the German sentence, print out the text (with start and end tokens) as well as the tokenized sequence.
* Pad the end of the tokenized German sequences with zeros, and batch the complete set of sequences into one numpy array.

In [4]:
import numpy as np

English = []
German  = []

for sentence in data_examples:
    s = sentence.split('\t')

    s[0] = preprocess_sentence(s[0])
    s[1] = preprocess_sentence(s[1])
    
    English.append(s[0])
    German.append("<start> " + s[1] + " <end>")
    
English = np.asarray(English)
German = np.asarray(German)
print("English: ", len(English))
print("German: ", len(German))


English:  20000
German:  20000


In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = '')
tokenizer.fit_on_texts(German)    
sequences = tokenizer.texts_to_sequences(German)
sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')

In [6]:
elements = random.sample(range(len(German)), 5)

for element in elements:
    print('English: ', English[element], '\n', 
          'German: ', German[element], '\n', 
          'Embedding: ', sequences[element], '\n')

English:  i'm . 
 German:  <start> ich bin . <end> 
 Embedding:  [ 1  4 15  3  2  0  0  0  0  0  0  0  0  0] 

English:  don't be afraid . 
 German:  <start> habe keine angst . <end> 
 Embedding:  [  1  18 103 190   3   2   0   0   0   0   0   0   0   0] 

English:  he made her cry . 
 German:  <start> er brachte sie zum weinen . <end> 
 Embedding:  [  1  14 779   8 198 196   3   2   0   0   0   0   0   0] 

English:  tom is contented . 
 German:  <start> tom ist zufrieden . <end> 
 Embedding:  [  1   5   6 508   3   2   0   0   0   0   0   0   0   0] 

English:  tom isn't hurt . 
 German:  <start> tom ist unversehrt . <end> 
 Embedding:  [   1    5    6 2764    3    2    0    0    0    0    0    0    0    0] 



## 2. Prepare the data with tf.data.Dataset objects

#### Load the embedding layer
As part of the dataset preproceessing for this project, you will use a pre-trained English word embedding module from TensorFlow Hub. The URL for the module is https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1. This module has also been made available as a complete saved model in the folder `'./models/tf2-preview_nnlm-en-dim128_1'`. 

This embedding takes a batch of text tokens in a 1-D tensor of strings as input. It then embeds the separate tokens into a 128-dimensional space. 

The code to load and test the embedding layer is provided for you below.

**NB:** this model can also be used as a sentence embedding module. The module will process each token by removing punctuation and splitting on spaces. It then averages the word embeddings over a sentence to give a single embedding vector. However, we will use it only as a word embedding module, and will pass each word in the input sentence as a separate token.

In [7]:
# Load embedding module from Tensorflow Hub

embedding_layer = hub.KerasLayer("./models/tf2-preview_nnlm-en-dim128_1",  
                                 output_shape=[128], input_shape=[], dtype=tf.string)

In [8]:
# Test the layer

embedding_layer(tf.constant(["these", "aren't", "the", "droids", "you're", "looking", "for"])).shape

TensorShape([7, 128])

You should now prepare the training and validation Datasets.

* Create a random training and validation set split of the data, reserving e.g. 20% of the data for validation (NB: each English dataset example is a single sentence string, and each German dataset example is a sequence of padded integer tokens).
* Load the training and validation sets into a tf.data.Dataset object, passing in a tuple of English and German data for both training and validation sets.
* Create a function to map over the datasets that splits each English sentence at spaces. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.strings.split function._
* Create a function to map over the datasets that embeds each sequence of English words using the loaded embedding layer/model. Apply this function to both Dataset objects using the map method.
* Create a function to filter out dataset examples where the English sentence is more than 13 (embedded) tokens in length. Apply this function to both Dataset objects using the filter method.
* Create a function to map over the datasets that pads each English sequence of embeddings with some distinct padding value before the sequence, so that each sequence is length 13. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.pad function. You can extract a Tensor shape using tf.shape; you might also find the tf.math.maximum function useful._
* Batch both training and validation Datasets with a batch size of 16.
* Print the `element_spec` property for the training and validation Datasets. 
* Using the Dataset `.take(1)` method, print the shape of the English data example from the training Dataset.
* Using the Dataset `.take(1)` method, print the German data example Tensor from the validation Dataset.

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

In [10]:
# 80% Train / 20% Validation
size = int(0.8*len(English))

# Shuffle indices to randomly get training and validation indices
indices = np.random.permutation(len(English))
training_idx, val_idx = indices[:size], indices[size:]

#Get train/val sets for English sentences and German sequences
train_E, val_E = English[training_idx], English[val_idx]
train_seq, val_seq = sequences[training_idx], sequences[val_idx]


In [11]:
#Create Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_E, train_seq))
val_dataset = tf.data.Dataset.from_tensor_slices((val_E, val_seq))


In [12]:
#Split sentences on spaces
def split_spaces(sentence, sequence):
    s = tf.strings.split(sentence)
    
    return s, sequence

train_dataset = train_dataset.map(split_spaces)
val_dataset = val_dataset.map(split_spaces)


In [13]:
#Use embedding layer from hub to encode word list
def embed(word_list, sequence):
    e = embedding_layer(word_list)
    
    return e, sequence

train_dataset = train_dataset.map(embed)
val_dataset = val_dataset.map(embed)


In [ ]:
#Filter out English sequences longer than 13
train_dataset = train_dataset.filter(lambda x,_: len(x)<13)
val_dataset = val_dataset.filter(lambda x,_: len(x)<13)  

print("Training Dataset: ")
print(train_dataset.element_spec)
print("Validation Dataset: ")
print(val_dataset.element_spec)

Training Dataset: 
(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(14,), dtype=tf.int32, name=None))
Validation Dataset: 
(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(14,), dtype=tf.int32, name=None))


In [ ]:
#Pad English Sequences to length 13
def pad_seq(eng_seq, ger_seq):
    
    #Find sequence length
    paddings = [[13-len(eng_seq),0],[0,0]]

    p = tf.pad(eng_seq, paddings=paddings, constant_values=0)
        
    return p, ger_seq
        
    
train_dataset = train_dataset.map(pad_seq)
val_dataset = val_dataset.map(pad_seq) 

In [ ]:
train_dataset = train_dataset.batch(16, drop_remainder=False)
val_dataset = val_dataset.batch(16, drop_remainder=False)


In [ ]:
print("Training Dataset: ")
print(train_dataset.element_spec)
print("Validation Dataset: ")
print(val_dataset.element_spec)

Training Dataset: 
(TensorSpec(shape=(None, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(None, 14), dtype=tf.int32, name=None))
Validation Dataset: 
(TensorSpec(shape=(None, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(None, 14), dtype=tf.int32, name=None))


In [ ]:
for english_train_example, german_train_example in train_dataset.take(1):
    print("English Training Example: ", english_train_example)
    print("German Training Example: ", german_train_example)
# get the shape for a german sequence example in the validation dataset
for english_val_example, german_val_example in val_dataset.take(1):
    print("English Validation Example: ", english_val_example)
    print("German Training Example: ",german_val_example)

English Training Example:  tf.Tensor(
[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.14532246 -0.08859951 -0.06054637 ...  0.03215049 -0.13606186
    0.09750178]
  [ 0.04849293  0.00719573 -0.12629282 ... -0.13178639 -0.14557579
   -0.04878096]
  [ 0.012986    0.08981702  0.16017003 ...  0.06796802  0.13528903
   -0.022035  ]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.0927744  -0.02721977 -0.06013793 ... -0.0429398   0.01172086
   -0.1628441 ]
  [ 0.09180953  0.00573924 -0.00290855 ... -0.11794681  0.07120994
   -0.04768806]
  [ 0.012986    0.08981702  0.16017003 ...  0.06796802  0.13528903
   -0.022035  ]]

## 3. Create the custom layer
You will now create a custom layer to add the learned end token embedding to the encoder model:

![Encoder schematic](data/neural_translation_model_encoder.png)

You should now build the custom layer.
* Using layer subclassing, create a custom layer that takes a batch of English data examples from one of the Datasets, and adds a learned embedded ‘end’ token to the end of each sequence. 
* This layer should create a TensorFlow Variable (that will be learned during training) that is 128-dimensional (the size of the embedding space). _Hint: you may find it helpful in the call method to use the tf.tile function to replicate the end token embedding across every element in the batch._
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and print the shape. Test the custom layer by calling the layer on the English data batch Tensor and print the resulting Tensor shape (the layer should increase the sequence length by one).

In [ ]:
num_tokens = len(tokenizer.word_index) + 1
class EndTokenLayer(Layer):
    
    def __init__(self, embedding_dim=128, **kwargs):
        super(EndTokenLayer, self).__init__(**kwargs)
        self.embedding_dim = embedding_dim
    
    def build(self, input_shape):
        self.end_token_emb = self.add_weight(shape=(input_shape[-1],), initializer='random_uniform', trainable= True)
  
    def call(self, inputs):
        end_token = tf.tile(tf.reshape(self.end_token_emb, shape=(1, 1, self.end_token_emb.shape[0])), 
                            [tf.shape(inputs)[0],1,1])
    
        return tf.keras.layers.concatenate([inputs, end_token], axis=1)

In [ ]:
endlayer = EndTokenLayer()
for english,german in train_dataset.take(1):
    temp_layer = endlayer(english)
    print("english sentences shape")
    print(english.shape) 

english sentences shape
(16, 13, 128)


## 4. Build the encoder network
The encoder network follows the schematic diagram above. You should now build the RNN encoder model.
* Using the functional API, build the encoder network according to the following spec:
    * The model will take a batch of sequences of embedded English words as input, as given by the Dataset objects.
    * The next layer in the encoder will be the custom layer you created previously, to add a learned end token embedding to the end of the English sequence.
    * This is followed by a Masking layer, with the `mask_value` set to the distinct padding value you used when you padded the English sequences with the Dataset preprocessing above.
    * The final layer is an LSTM layer with 512 units, which also returns the hidden and cell states.
    * The encoder is a multi-output model. There should be two output Tensors of this model: the hidden state and cell states of the LSTM layer. The output of the LSTM layer is unused.
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and test the encoder model by calling it on the English data Tensor, and print the shape of the resulting Tensor outputs.
* Print the model summary for the encoder network.

In [ ]:
def Encoder(input_shape):
    inputs = Input([13,input_shape])
    etl = EndTokenLayer()(inputs)
    mask = Masking([(lambda x: x*0)(x) for x in range(128)])(etl)
    lstm , hidden_state, cell_state = LSTM(512, return_sequences=True, return_state=True)(mask)
  
    model = Model(inputs=inputs, outputs=[hidden_state, cell_state])

    return model

In [ ]:
encoder_model = Encoder(128)
encoder_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13, 128)]         0         
_________________________________________________________________
end_token_layer_1 (EndTokenL (None, 14, 128)           128       
_________________________________________________________________
masking (Masking)            (None, 14, 128)           0         
_________________________________________________________________
lstm (LSTM)                  [(None, 14, 512), (None,  1312768   
Total params: 1,312,896
Trainable params: 1,312,896
Non-trainable params: 0
_________________________________________________________________


## 5. Build the decoder network
The decoder network follows the schematic diagram below. 

![Decoder schematic](data/neural_translation_model_decoder.png)

You should now build the RNN decoder model.
* Using Model subclassing, build the decoder network according to the following spec:
    * The initializer should create the following layers:
        * An Embedding layer with vocabulary size set to the number of unique German tokens, embedding dimension 128, and set to mask zero values in the input.
        * An LSTM layer with 512 units, that returns its hidden and cell states, and also returns sequences.
        * A Dense layer with number of units equal to the number of unique German tokens, and no activation function.
    * The call method should include the usual `inputs` argument, as well as the additional keyword arguments `hidden_state` and `cell_state`. The default value for these keyword arguments should be `None`.
    * The call method should pass the inputs through the Embedding layer, and then through the LSTM layer. If the `hidden_state` and `cell_state` arguments are provided, these should be used for the initial state of the LSTM layer. _Hint: use the_ `initial_state` _keyword argument when calling the LSTM layer on its input._
    * The call method should pass the LSTM output sequence through the Dense layer, and return the resulting Tensor, along with the hidden and cell states of the LSTM layer.
* Using the Dataset `.take(1)` method, extract a batch of English and German data examples from the training Dataset. Test the decoder model by first calling the encoder model on the English data Tensor to get the hidden and cell states, and then call the decoder model on the German data Tensor and hidden and cell states, and print the shape of the resulting decoder Tensor outputs.
* Print the model summary for the decoder network.

In [ ]:
num_tokens = len(tokenizer.word_index) + 1

In [ ]:
class DecoderModel(Model):
    def __init__(self,initial_state=True,**kwargs):
        super(DecoderModel, self).__init__(**kwargs)
        self.embedding = Embedding(input_dim = num_tokens,output_dim = 128,mask_zero = True)
        self.lstm = LSTM(512, return_sequences=True, return_state=True)
        self.dense = Dense(num_tokens)
        self.initial_state = initial_state

    def call(self,inputs,hidden_state = None,cell_state = None):
        h = self.embedding(inputs)
    
        lstm,hidden_1,cell_1 = self.lstm(h,initial_state = [hidden_state,cell_state])        
        h = self.dense(lstm)
    
        return h,hidden_1,cell_1

In [ ]:
decoder_model = DecoderModel()

In [ ]:
#or english,german in train_dataset.take(1):
#    temp,hidden_1,cell_1 = decoder_model(german)
#print("decoder output shape:")
#print(tf.shape(temp))
#print("hidden state shape:")
#print(tf.shape(hidden_1))
#print("cell state shape:")
#print(tf.shape(cell_1))

decoder_model = DecoderModel()
# test the decoder model 
for english, german in train_dataset.take(1):
    hidden, cell_state = encoder_model(english)
    output, hidden, cell_state = decoder_model(german, hidden, cell_state)
    print(f"output.shape: {output.shape}")
    print(f"hidden.shape: {hidden.shape}")
    print(f"cell_state.shape: {cell_state.shape}")

# model summary 
decoder_model.summary()

output.shape: (16, 14, 5744)
hidden.shape: (16, 512)
cell_state.shape: (16, 512)
Model: "decoder_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  735232    
_________________________________________________________________
lstm_2 (LSTM)                multiple                  1312768   
_________________________________________________________________
dense_1 (Dense)              multiple                  2946672   
Total params: 4,994,672
Trainable params: 4,994,672
Non-trainable params: 0
_________________________________________________________________


## 6. Make a custom training loop
You should now write a custom training loop to train your custom neural translation model.
* Define a function that takes a Tensor batch of German data (as extracted from the training Dataset), and returns a tuple containing German inputs and outputs for the decoder model (refer to schematic diagram above).
* Define a function that computes the forward and backward pass for your translation model. This function should take an English input, German input and German output as arguments, and should do the following:
    * Pass the English input into the encoder, to get the hidden and cell states of the encoder LSTM.
    * These hidden and cell states are then passed into the decoder, along with the German inputs, which returns a sequence of outputs (the hidden and cell state outputs of the decoder LSTM are unused in this function).
    * The loss should then be computed between the decoder outputs and the German output function argument.
    * The function returns the loss and gradients with respect to the encoder and decoder’s trainable variables.
    * Decorate the function with @tf.function
* Define and run a custom training loop for a number of epochs (for you to choose) that does the following:
    * Iterates through the training dataset, and creates decoder inputs and outputs from the German sequences.
    * Updates the parameters of the translation model using the gradients of the function above and an optimizer object.
    * Every epoch, compute the validation loss on a number of batches from the validation and save the epoch training and validation losses.
* Plot the learning curves for loss vs epoch for both training and validation sets.

_Hint: This model is computationally demanding to train. The quality of the model or length of training is not a factor in the grading rubric. However, to obtain a better model we recommend using the GPU accelerator hardware on Colab._

In [ ]:
def german_data(german_data):
#    input_data = german_data[:,0:tf.shape(german_data)[1]-1]
#    output_data = german_data[:,1:tf.shape(german_data)[1]]
    input_data = german_data[:, :-1]
    output_data = german_data[:, 1:]

    return input_data, output_data


In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
@tf.function
def grad_fn(english, german_input, german_output, loss):
    with tf.GradientTape() as tape:
        hidden, cell_state = encoder_model(english)
        german_from_model, hidden, cell_state = decoder_model(german_input, hidden, cell_state)
        
        loss = tf.math.reduce_mean(loss(german_output, german_from_model))
    
        grad_encoder, grad_decoder = tape.gradient(loss, [encoder_model.trainable_variables,
                                                      decoder_model.trainable_variables])        
        
        return loss, grad_encoder, grad_decoder

In [ ]:
num_epochs = 3
epoch_train_loss = []
epoch_val_loss = []

for epoch in range(num_epochs):
    train_loss = tf.keras.metrics.Mean()
    val_loss = tf.keras.metrics.Mean()

    #training loop 
    for english, german in train_dataset:
        # create the german input and output 
        german_in, german_out = german_data(german)
        
        #optimize the model
        loss_value, grad_encoder, grad_decoder = grad_fn(english, german_in, german_out, loss)
        optimizer.apply_gradients(zip(grad_encoder, encoder_model.trainable_variables))
        optimizer.apply_gradients(zip(grad_decoder, decoder_model.trainable_variables))
    
        #compute current loss
        train_loss(loss_value)
        
    epoch_train_loss.append(train_loss.result())

    #validation loop
    for english, german in val_dataset:
        # create the german input and output 
        german_in, german_out = german_data(german)
    
        # put it through the model
        hidden, cell_state = encoder_model(english)
        german_from_model, hidden, cell_state = decoder_model(german_in, hidden, cell_state)
        
        loss = tf.keras.losses.sparse_categorical_crossentropy(german_out, 
                                                           german_from_model,
                                                           from_logits=True)
        val_loss(loss)
    
    epoch_val_loss.append(val_loss.result())
    
    print(f"Epoch {epoch};\t Loss train {epoch_train_loss[-1]};\tLoss val {epoch_val_loss[-1]}")

tf.Tensor(8.656017, shape=(), dtype=float32)
tf.Tensor(8.6417, shape=(), dtype=float32)
tf.Tensor(8.623888, shape=(), dtype=float32)
tf.Tensor(8.59469, shape=(), dtype=float32)
tf.Tensor(8.531662, shape=(), dtype=float32)
tf.Tensor(8.44398, shape=(), dtype=float32)
tf.Tensor(8.140308, shape=(), dtype=float32)
tf.Tensor(7.5355124, shape=(), dtype=float32)
tf.Tensor(6.3403416, shape=(), dtype=float32)
tf.Tensor(5.1943107, shape=(), dtype=float32)
tf.Tensor(3.829519, shape=(), dtype=float32)
tf.Tensor(3.4167705, shape=(), dtype=float32)
tf.Tensor(3.0955012, shape=(), dtype=float32)
tf.Tensor(3.541086, shape=(), dtype=float32)
tf.Tensor(2.8554387, shape=(), dtype=float32)
tf.Tensor(3.1895802, shape=(), dtype=float32)
tf.Tensor(2.7460573, shape=(), dtype=float32)
tf.Tensor(3.0485752, shape=(), dtype=float32)
tf.Tensor(2.9542668, shape=(), dtype=float32)
tf.Tensor(3.3914897, shape=(), dtype=float32)
tf.Tensor(3.3918774, shape=(), dtype=float32)
tf.Tensor(2.866859, shape=(), dtype=float32)
tf

tf.Tensor(2.0488765, shape=(), dtype=float32)
tf.Tensor(1.7600793, shape=(), dtype=float32)
tf.Tensor(1.9069173, shape=(), dtype=float32)
tf.Tensor(1.6109531, shape=(), dtype=float32)
tf.Tensor(1.7972233, shape=(), dtype=float32)
tf.Tensor(1.7538332, shape=(), dtype=float32)
tf.Tensor(1.8937106, shape=(), dtype=float32)
tf.Tensor(1.7560086, shape=(), dtype=float32)
tf.Tensor(1.6728463, shape=(), dtype=float32)
tf.Tensor(1.8788091, shape=(), dtype=float32)
tf.Tensor(1.6316602, shape=(), dtype=float32)
tf.Tensor(1.8947055, shape=(), dtype=float32)
tf.Tensor(1.9129156, shape=(), dtype=float32)
tf.Tensor(1.8706838, shape=(), dtype=float32)
tf.Tensor(1.7290391, shape=(), dtype=float32)
tf.Tensor(1.9286467, shape=(), dtype=float32)
tf.Tensor(1.8459779, shape=(), dtype=float32)
tf.Tensor(1.6250261, shape=(), dtype=float32)
tf.Tensor(1.7745064, shape=(), dtype=float32)
tf.Tensor(1.6130122, shape=(), dtype=float32)
tf.Tensor(1.9896622, shape=(), dtype=float32)
tf.Tensor(1.4871653, shape=(), dty

tf.Tensor(1.5318127, shape=(), dtype=float32)
tf.Tensor(1.555042, shape=(), dtype=float32)
tf.Tensor(1.5270486, shape=(), dtype=float32)
tf.Tensor(1.5716236, shape=(), dtype=float32)
tf.Tensor(1.487577, shape=(), dtype=float32)
tf.Tensor(1.7015465, shape=(), dtype=float32)
tf.Tensor(1.5818362, shape=(), dtype=float32)
tf.Tensor(1.5232213, shape=(), dtype=float32)
tf.Tensor(1.7429379, shape=(), dtype=float32)
tf.Tensor(1.5027997, shape=(), dtype=float32)
tf.Tensor(1.7999918, shape=(), dtype=float32)
tf.Tensor(1.4426246, shape=(), dtype=float32)
tf.Tensor(1.7631367, shape=(), dtype=float32)
tf.Tensor(1.519506, shape=(), dtype=float32)
tf.Tensor(1.570518, shape=(), dtype=float32)
tf.Tensor(1.6303624, shape=(), dtype=float32)
tf.Tensor(1.5753939, shape=(), dtype=float32)
tf.Tensor(1.4888515, shape=(), dtype=float32)
tf.Tensor(1.6945623, shape=(), dtype=float32)
tf.Tensor(1.6420943, shape=(), dtype=float32)
tf.Tensor(1.6175145, shape=(), dtype=float32)
tf.Tensor(1.7423286, shape=(), dtype=f

tf.Tensor(1.5303218, shape=(), dtype=float32)
tf.Tensor(1.3571709, shape=(), dtype=float32)
tf.Tensor(1.7390888, shape=(), dtype=float32)
tf.Tensor(1.7203138, shape=(), dtype=float32)
tf.Tensor(1.6181058, shape=(), dtype=float32)
tf.Tensor(1.5036772, shape=(), dtype=float32)
tf.Tensor(1.4307424, shape=(), dtype=float32)
tf.Tensor(1.5595422, shape=(), dtype=float32)
tf.Tensor(1.4328109, shape=(), dtype=float32)
tf.Tensor(1.5370326, shape=(), dtype=float32)
tf.Tensor(1.4236203, shape=(), dtype=float32)
tf.Tensor(1.5722936, shape=(), dtype=float32)
tf.Tensor(1.4188142, shape=(), dtype=float32)
tf.Tensor(1.4027051, shape=(), dtype=float32)
tf.Tensor(1.4913564, shape=(), dtype=float32)
tf.Tensor(1.5997882, shape=(), dtype=float32)
tf.Tensor(1.510003, shape=(), dtype=float32)
tf.Tensor(1.36078, shape=(), dtype=float32)
tf.Tensor(1.4025172, shape=(), dtype=float32)
tf.Tensor(1.6678777, shape=(), dtype=float32)
tf.Tensor(1.6514373, shape=(), dtype=float32)
tf.Tensor(1.5817765, shape=(), dtype=

## 7. Use the model to translate
Now it's time to put your model into practice! You should run your translation for five randomly sampled English sentences from the dataset. For each sentence, the process is as follows:
* Preprocess and embed the English sentence according to the model requirements.
* Pass the embedded sentence through the encoder to get the encoder hidden and cell states.
* Starting with the special  `"<start>"` token, use this token and the final encoder hidden and cell states to get the one-step prediction from the decoder, as well as the decoder’s updated hidden and cell states.
* Create a loop to get the next step prediction and updated hidden and cell states from the decoder, using the most recent hidden and cell states. Terminate the loop when the `"<end>"` token is emitted, or when the sentence has reached a maximum length.
* Decode the output token sequence into German text and print the English text and the model's German translation.